<a href="https://colab.research.google.com/github/Thrillcrazyer/Deep-Learning-with-TensorFlow-2-and-Keras-in-colab-/blob/master/chap05/Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tf.keras 추정기 모델로 패션 MNIST 분류

In [11]:
import os
import time
import tensorflow as tf
import numpy as np

#예측하려는 범주의 수 (0~9)

LABEL_DIMENSIONS= 10

(train_images, train_labels),(test_images, test_labels)= tf.keras.datasets.fashion_mnist.load_data()
TRAINING_SIZE= len(train_images)
TEST_SIZE=len(test_images)


#훈련이미지를 변환하고 채널을 추가
train_images = np.asarray(train_images,dtype=np.float32)/255 
train_images=train_images.reshape(TRAINING_SIZE,28,28,1)

#테스트 이미지를 변환하고 체널을 추가 
test_images=np.asarray(test_images,dtype=np.float32)/255
test_images=test_images.reshape(TEST_SIZE,28,28,1)

train_labels=tf.keras.utils.to_categorical(train_labels,LABEL_DIMENSIONS)
test_labels=tf.keras.utils.to_categorical(test_labels,LABEL_DIMENSIONS)

#레이블을 float 형식으로 변환
train_labels= train_labels.astype(np.float32)
test_labels= test_labels.astype(np.float32)
print(train_labels.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


In [12]:
# GPU 사용하는거

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [13]:
#tf.Keras 함수형 API로 모델 구축 (Sequantial과 다른형태의 모델링)

inputs=tf.keras.Input(shape=(28,28,1))
x=tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')(inputs)
x=tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)
x=tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu')(inputs)
x=tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)
x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
x=tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

predictions=tf.keras.layers.Dense(LABEL_DIMENSIONS,activation='softmax')(x)

model=tf.keras.Model(inputs=inputs,outputs=predictions)
model.summary()




Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 256)       147712    
                                                                 
 flatten_3 (Flatten)         (None, 30976)             0         
                                                                 
 dense_8 (Dense)             (None, 64)                1982528   
                                                           

In [14]:
##optimizer= tf.keras.optimizers.SGD()

#model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy',
#              optimizer=optimizer,
#              metrics=['accuracy'])

optimizer = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

strategy=None
#strategy= tf.distribute.MirroredStrategy()
config= tf.estimator.RunConfig(train_distribute=strategy)

estimator = tf.keras.estimator.model_to_estimator(model, config=config)

def input_fn(images, labels, epochs, batch_size):
  #입력을 데이터셋으로 변환
  dataset=tf.data.Dataset.from_tensor_slices((images,labels))

  #예시를 섞고, 반복하고, 배치한다.
  SHUFFLE_SIZE= 5000
  dataset= dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
  dataset = dataset.prefetch(None)

  #데이터셋 반환
  return dataset

INFO:tensorflow:Using the Keras model provided.


/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp5_gp42ab', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
# 훈련및 평가

# 훈련
BATCH_SIZE=512
EPOCHS=50

estimator_train_result = estimator.train(input_fn=lambda:input_fn(train_images,train_labels,epochs=EPOCHS, batch_size= BATCH_SIZE))

print(estimator_train_result)

#평가
estimator.evaluate(lambda:input_fn(test_images,test_labels,epochs=1, batch_size=BATCH_SIZE))

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmp5_gp42ab/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /tmp/tmp5_gp42ab/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 10 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp5_gp42ab/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.302271, step = 0
INFO:tensorflow:global_step/sec: 39.3514
INFO:tensorflow:loss = 0.42053318, step = 100 (2.547 sec)
INFO:tensorflow:global_

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


INFO:tensorflow:Inference Time : 0.72371s
INFO:tensorflow:Finished evaluation at 2022-07-06-09:24:24
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.928, global_step = 5860, loss = 0.4667523
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /tmp/tmp5_gp42ab/model.ckpt-5860


{'accuracy': 0.928, 'global_step': 5860, 'loss': 0.4667523}